# KerasCNN3

In [1]:
import numpy as np
import pandas as pd
import keras as k
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

Using TensorFlow backend.


In [2]:
train = pd.read_json('../ShipIceberg/Data/train.json')
train.inc_angle = train.inc_angle.replace('na',0)

In [3]:
def transform(df):
    images = []
    for i, row in df.iterrows():
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
    
        # Feature scaling
        band_1_norm = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        band_2_norm = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        images.append(np.dstack((band_1_norm, band_2_norm)))
    return np.array(images)

def augment(images):
    image_rot90 = []
    image_rot180 = []
    image_rot270 = []
    image_mirror_lr = []
    image_mirror_ud = []
    for i in range(0,images.shape[0]):
        band_1 = images[i,:,:,0]
        band_2 = images[i,:,:,1]
        
        # rotate 90
        band_1_rot90 = np.rot90(band_1)
        band_2_rot90 = np.rot90(band_2)
        image_rot90.append(np.dstack((band_1_rot90, band_2_rot90)))
        
        # rotate 180
        band_1_rot180 = np.rot90(band_1_rot90)
        band_2_rot180 = np.rot90(band_2_rot90)
        image_rot180.append(np.dstack((band_1_rot180, band_2_rot180)))
        
        # rotate 270
        band_1_rot270 = np.rot90(band_1_rot180)
        band_2_rot270 = np.rot90(band_2_rot180)
        image_rot270.append(np.dstack((band_1_rot270, band_2_rot270)))
        
        # mirror left-right
        band_1_mirror_lr = np.flip(band_1, 0)
        band_2_mirror_lr = np.flip(band_2, 0)
        image_mirror_lr.append(np.dstack((band_1_mirror_lr, band_2_mirror_lr)))
        
        # mirror up-down
        band_1_mirror_ud = np.flip(band_1, 1)
        band_2_mirror_ud = np.flip(band_2, 1)
        image_mirror_ud.append(np.dstack((band_1_mirror_ud, band_2_mirror_ud)))
        
    rot90 = np.array(image_rot90)
    rot180 = np.array(image_rot180)
    rot270 = np.array(image_rot270)
    mirrorlr = np.array(image_mirror_lr)
    mirrorud = np.array(image_mirror_ud)
    images = np.concatenate((images, rot90, rot180, rot270, mirrorlr, mirrorud))
    return images

In [4]:
train_X = transform(train)
train_y = np.array(train['is_iceberg'])
train_X = augment(train_X)
train_y = np.concatenate((train_y, train_y, train_y, train_y, train_y, train_y))

In [5]:
from sklearn.model_selection import train_test_split

X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(train_X, 
                                                            train_y, 
                                                            random_state=1, 
                                                            train_size=0.8)

In [6]:
model=k.models.Sequential()

model.add(k.layers.convolutional.Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 2)))
model.add(k.layers.convolutional.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(k.layers.convolutional.Conv2D(64, kernel_size=(3, 3),activation='relu'))
model.add(k.layers.convolutional.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(k.layers.convolutional.Conv2D(128, kernel_size=(3, 3),activation='relu'))
model.add(k.layers.convolutional.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(k.layers.convolutional.Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(k.layers.convolutional.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(k.layers.Dropout(0.25))

model.add(k.layers.Flatten())

model.add(k.layers.Dense(256, activation='relu'))
model.add(k.layers.Dropout(0.3))

model.add(k.layers.Dense(128, activation='relu'))
model.add(k.layers.Dropout(0.4))

model.add(k.layers.Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer = k.optimizers.Nadam(lr=0.001), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 32)        608       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 5, 256)         295168    
__________

In [7]:
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

model.fit(X_train_cv, y_train_cv, batch_size=100, epochs=25, verbose=1, 
          validation_data=(X_valid, y_valid),callbacks=callbacks)

Train on 7699 samples, validate on 1925 samples
Epoch 1/25
7699/7699 [==============================] - 17s - loss: 0.6500 - acc: 0.6080 - val_loss: 0.5107 - val_acc: 0.7325
Epoch 2/25
7699/7699 [==============================] - 5s - loss: 0.4858 - acc: 0.7657 - val_loss: 0.3489 - val_acc: 0.8322
Epoch 3/25
7699/7699 [==============================] - 5s - loss: 0.3586 - acc: 0.8448 - val_loss: 0.3159 - val_acc: 0.8603
Epoch 4/25
7699/7699 [==============================] - 5s - loss: 0.2985 - acc: 0.8714 - val_loss: 0.2953 - val_acc: 0.8649
Epoch 5/25
7699/7699 [==============================] - 5s - loss: 0.2802 - acc: 0.8810 - val_loss: 0.2717 - val_acc: 0.8779
Epoch 6/25
7699/7699 [==============================] - 5s - loss: 0.2421 - acc: 0.9009 - val_loss: 0.2542 - val_acc: 0.8836
Epoch 7/25
7699/7699 [==============================] - 5s - loss: 0.2228 - acc: 0.9125 - val_loss: 0.2679 - val_acc: 0.8816
Epoch 8/25
7699/7699 [==============================] - 5s - loss: 0.1969 - 

In [9]:
model.load_weights(filepath=file_path)
score = model.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

1925/1925 [==============================] - 20s    
Test loss: 0.254180503015
Test accuracy: 0.883636363636


In [8]:
#test = pd.read_json('../input/test.json')
import json
with open('../ShipIceberg/Data/test.json', 'r') as f:
    test = json.load(f)
    test=pd.DataFrame(test)

test.inc_angle = test.inc_angle.replace('na',0)
test_X = transform(test)
pred_test = model.predict(test_X)
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': pred_test.reshape((pred_test.shape[0]))})
submission.to_csv('iceberg_kaggle.csv', index=False)